In [1]:
import pandas as pd
import dash
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output

In [8]:
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})

In [12]:
JupyterDash.infer_jupyter_proxy_config()

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

colors = {'background': '#111111', 'text': '#7FDBFF'}

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(style={'backgroundColor': colors['background']}, children = [
                        html.H1("Airline Performance Dashboard", 
                                style={'font-size':30, 'textAlign':'center', 'color': colors['text']}),
                       html.Div(["Input Year: ", dcc.Input(id='input-year', value='2010', type='number',
                                                          style={'height':'25px', 'font-size':20, 'color': colors['text']})],
                               style={'font-size':25, 'color': colors['text']}),
                       html.Br(),
                       html.Br(),
                       html.Div( 
                            [
                           html.Div(dcc.Graph(id='carrier-plot')),
                           html.Div(dcc.Graph(id='weather-plot'))], 
                           style={'display':'flex', 'backgroundColor': colors['background']}),
                       html.Div( 
                            [
                           html.Div(dcc.Graph(id='nas-plot')),
                           html.Div(dcc.Graph(id='security-plot'))], 
                           style={'display':'flex', 'backgroundColor': colors['background']}),
                       html.Div(dcc.Graph(id='late-plot'), style={'width':'65%', 'backgroundColor': colors['background']})
                      ])

""" Compute_info function description

This function takes in airline data and selected year as an input and performs computation for creating charts and plots.

Arguments:
    airline_data: Input airline data.
    entered_year: Input year for which computation needs to be performed.

Returns:
    Computed average dataframes for carrier delay, weather delay, NAS delay, security delay, and late aircraft delay.

"""

def compute_info(airline_data, entered_year):
    # Select data
    df = airline_data[airline_data['Year'] == int(entered_year)]
    # Compute delay averages
    avg_carrier = df.groupby(['Month', 'Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month', 'Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_nas = df.groupby(['Month', 'Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month', 'Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month', 'Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    return avg_carrier, avg_weather, avg_nas, avg_sec, avg_late

"""Callback Function

Function that returns fugures using the provided input year.

Arguments:

    entered_year: Input year provided by the user.
    
Returns:

    List of figures computed using the provided helper function `compute_info`.
"""

@app.callback([
                Output(component_id='carrier-plot', component_property='figure'),
                Output(component_id='weather-plot', component_property='figure'),
                Output(component_id='nas-plot', component_property='figure'),
                Output(component_id='security-plot', component_property='figure'),
                Output(component_id='late-plot', component_property='figure')
                ],
                Input(component_id='input-year', component_property='value'))

# Computation to callback function and return graph
def get_graph(entered_year):
    # Compute required information for creating graph from the data
    avg_carrier, avg_weather, avg_nas, avg_sec, avg_late = compute_info(airline_data, entered_year)
    # Line plot for carrier delay
    carrier_fig = px.line(avg_carrier, x='Month', y='CarrierDelay', color='Reporting_Airline', title='Average carrier delay time (minutes) by airline')
    carrier_fig.update_layout(plot_bgcolor=colors['background'], paper_bgcolor=colors['background'], font_color=colors['text'])
    # Line plot for weather delay
    weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline', title='Average weather delay time (minutes) by airline')
    weather_fig.update_layout(plot_bgcolor=colors['background'], paper_bgcolor=colors['background'], font_color=colors['text'])
    # Line plot for nas delay
    nas_fig = px.line(avg_nas, x='Month', y='NASDelay', color='Reporting_Airline', title='Average NAS delay time (minutes) by airline')
    nas_fig.update_layout(plot_bgcolor=colors['background'], paper_bgcolor=colors['background'], font_color=colors['text'])
    # Line plot for security delay
    sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay', color='Reporting_Airline', title='Average security delay time (minutes) by airline')
    sec_fig.update_layout(plot_bgcolor=colors['background'], paper_bgcolor=colors['background'], font_color=colors['text'])
    # Line plot for late aircraft delay
    late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline', title='Average late aircraft delay time (minutes) by airline')
    late_fig.update_layout(plot_bgcolor=colors['background'], paper_bgcolor=colors['background'], font_color=colors['text'])

    return [carrier_fig, weather_fig, nas_fig, sec_fig, late_fig]

if __name__ == '__main__':
    app.run_server(mode='external')
    

Dash app running on http://127.0.0.1:8050/
